In [ ]:
import os
import cv2
import numpy as np

IMG_SIZE = 256

def load_data(image_dir, mask_dir):
    images = []
    masks = []

    for file in os.listdir(image_dir):
        img = cv2.imread(os.path.join(image_dir, file), 0)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0

        mask = cv2.imread(os.path.join(mask_dir, file), 0)
        mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE))
        mask = mask / 255.0

        images.append(img)
        masks.append(mask)

    return np.array(images)[..., np.newaxis], np.array(masks)[..., np.newaxis]


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from utils import load_data

def build_unet(input_shape=(256,256,1)):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D()(c1)

    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D()(c2)

    # Bottleneck
    b = layers.Conv2D(256, 3, activation='relu', padding='same')(p2)
    b = layers.Conv2D(256, 3, activation='relu', padding='same')(b)

    # Decoder
    u1 = layers.UpSampling2D()(b)
    u1 = layers.concatenate([u1, c2])
    c3 = layers.Conv2D(128, 3, activation='relu', padding='same')(u1)
    c3 = layers.Conv2D(128, 3, activation='relu', padding='same')(c3)

    u2 = layers.UpSampling2D()(c3)
    u2 = layers.concatenate([u2, c1])
    c4 = layers.Conv2D(64, 3, activation='relu', padding='same')(u2)
    c4 = layers.Conv2D(64, 3, activation='relu', padding='same')(c4)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c4)

    return Model(inputs, outputs)

# Load data
X, Y = load_data("data/images", "data/masks")

model = build_unet()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X, Y, batch_size=8, epochs=20, validation_split=0.2)

model.save("unet_model.h5")


In [ ]:
model = tf.keras.models.load_model("unet_model.h5")

pred_mask = model.predict(X)
pred_mask = (pred_mask > 0.5).astype(np.uint8)


In [ ]:
import numpy as np
import cv2
from skimage.measure import label, regionprops

MICRONS_PER_PIXEL = 0.5  # adjust to your calibration

def extract_features(mask):

    labeled = label(mask)
    regions = regionprops(labeled)

    areas = []
    hex_count = 0

    for region in regions:
        area_pixels = region.area
        area_um = area_pixels * (MICRONS_PER_PIXEL**2)
        areas.append(area_um)

        # Hexagon detection
        cell_mask = (labeled == region.label).astype(np.uint8)
        contours, _ = cv2.findContours(cell_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0:
            cnt = contours[0]
            epsilon = 0.02 * cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, epsilon, True)
            if len(approx) == 6:
                hex_count += 1

    areas = np.array(areas)

    mean_area = np.mean(areas)
    std_area = np.std(areas)

    CV = std_area / mean_area
    density = len(areas) / (mask.shape[0] * mask.shape[1] * (MICRONS_PER_PIXEL**2))
    H = (hex_count / len(areas)) * 100

    return density, CV, H


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Example dataset structure:
# image_name | density | CV | H | label

df = pd.read_csv("features.csv")

X = df[['density','CV','H']]
y = df['label']  # 0 normal, 1 abnormal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

print("Accuracy:", clf.score(X_test, y_test))


In [ ]:
# Load model
unet = tf.keras.models.load_model("unet_model.h5")
clf = joblib.load("classifier.pkl")

# Predict segmentation
mask = unet.predict(image)
mask = (mask > 0.5).astype(np.uint8)

# Extract features
density, CV, H = extract_features(mask)

# Predict normal/abnormal
prediction = clf.predict([[density, CV, H]])

print("Prediction:", "Abnormal" if prediction[0]==1 else "Normal")
